In [120]:
import pandas as pd

In [121]:
folder = ".\\input_files\\other_format\\"
file = '521369749_MedStar_National_Rehabilitation_Hospital_Standard Charges.xlsx'

df = pd.read_excel(folder + file, skiprows=5)

In [122]:
df_copy = df.copy()

In [123]:
df.rename(columns={
    'Charge Description Master': 'description',
    'Revenue Code': 'rev_code',
    'CPT/HCPCS': 'code'
}, inplace=True)

df['rev_code'] = df['rev_code'].astype(str)

df['rev_code'] = df['rev_code'].str.split('.').str[0]
df.loc[df['rev_code'] == 'nan', 'rev_code'] = ''


In [124]:
cols = df.columns.tolist()
id_vars = cols[:3]
value_vars = cols[3:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='standard_charge')

In [125]:
df.loc[df['standard_charge'] == 'NA*', 'standard_charge'] = pd.NA
df.dropna(subset='standard_charge', inplace=True)

In [126]:
df['payer'] = df['payer'].str.replace('Negotiated Charge', '')

In [127]:
df['code'] = df['code'].str.strip()

In [128]:
df.loc[df['code'].str.len() > 6, 'modifiers'] = df['code'].str[-2:]
df.loc[df['code'].str.len() > 6, 'hcpcs_cpt'] = df['code'].str[:-2]
df.loc[df['code'].str.len() == 5, 'hcpcs_cpt'] = df['code']

In [129]:
mapping = {
    'Discounted Cash Price*': 'cash',
    'De-Identified Minimum Negotiated Charge': 'min',
    'De-Identified Maximum Negotiated Charge': 'max',
    'Gross Charges': 'gross'
}

df['payer_category'] = df['payer'].map(mapping).fillna('payer')

In [142]:
non_null_cols = ['code', 'modifiers', 'hcpcs_cpt']
# Have to convert them to string first, which converts nulls to nan
df[non_null_cols] = df[non_null_cols].astype(str)

df.loc[df['code'] == 'nan', 'code'] = ''
df.loc[df['hcpcs_cpt'] == 'nan', 'hcpcs_cpt'] = ''
df.loc[df['modifiers'] == 'nan','modifiers'] = ''

In [144]:
ccn = {
    '521369749': '093025',
    '522218584': '090004'
}

ein = file.split('_')[0]
df['id'] = ccn[ein]

,description,rev_code,code,payer,standard_charge,modifiers,hcpcs_cpt,payer_category
193,CIPROFLOXACIN 200MG D5200ML(2),250,J0744,Gross Charges,68.44,,J0744,gross
26449,BC-ANKLE 3 OR MORE VW LEFT,320,73610LT,Kaiser Virginia Medicaid,102.64,LT,73610,payer
1977,CA CLUCONATE 1GM/50ML ISO,250,J0611,Aetna Medicare Advantage,42.77,,J0611,payer
13859,*CHOLESTYRAMINE 4/9G 1 PKT,250,,Cigna Healthspring,3.65,,,payer
38970,*INSULIN LISPRO 5UN(60) 3ML,250,,United Healthcare Community Plan Maryland,0.83,,,payer
42829,*PRIMIDONE 250MG TAB,250,,United Healthcare,5.06,,,payer
43665,*OXCARBAZEPINE 150MG TAB,250,,United Healthcare,6.1,,,payer
7949,MRI PELVIS WO W CONTRAST,610,,CareFirst,1060.4,,,payer
25742,BLADE OSC 24.4X4.5MM,272,,Kaiser Virginia Medicaid,18.9,,,payer
13531,CCP ANTIBODY,302,,Cigna Healthspring,144.13,,,payer
